# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686563


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:35,  3.40s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:05,  2.44s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:46,  1.80s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:34,  1.37s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:12<00:01,  3.52it/s]

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  3.56it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  3.76it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  4.31it/s]

not-logged-in-0ed3d164fd32cee0b7f7    0.003614
RockyGeo                              0.000112
Tlalka                                0.041773
not-logged-in-ec4ead1d8579ea060634    0.000224
saab14                                0.000197
kcarte13                              0.000092
mtajnai                               0.000776
justsomenoodles                       0.000988
not-logged-in-ee87b8e6b2975640b600    0.000115
danielrwik                            0.000124
raisamaduro                           0.002100
gabrielweiss                          0.008620
RaulE                                 0.000116
acapirala                             0.000157
rhondacard                            0.001831
Lavadude                              0.022517
clars915                              0.000141
not-logged-in-8700ff8251e0704f745c    0.031247
not-logged-in-8700ff8251e0704f745c    0.000084
TanlerP                               0.026223
equidad1                              0.000446
LeusaneLordel

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())